<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate the Fosberg Fire Weather Index for the UFS S2S</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 11/29/23
<br>
<br>    
<br>
Takes models/variables downloaded and calculates the Fosberg Fire Weather Index. This notebook leverages already generated database files.
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## Optional: Establish a dask client. This is a lot of data.

In [ ]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [ ]:
client = Client(Cluster)
client

### The integral notebook of functions to run

In [ ]:
%run File_concat_mod_functions.ipynb

## The Fosberg functions

In [1]:
def get_m(h, t):

    ''' This function calculates the equilibrium moisture content (m) for the Fosberg Index given conditionals.

    Inputs:

    h: (float) relative humidity value 
    t: (float) temperature value

    Outputs:

    m: (float or nan) 

    '''
    
    if h < 10:
    
        m =0.03229 + 0.281073*h - 0.000578*h*t
    
    elif h >= 10 and h < 50:
    
        m = 2.22749 + 0.160107*h - 0.01478*t
    
    elif h >= 50:
    
        m = 21.0606 + 0.005565*h**2 - 0.00035*h*t - 0.483199*h

    else:

        m = np.nan

    return m


# UFS 

In [ ]:
output_dir = 'database_files'

In [ ]:
model = 'UFS_S2S'
var_list = ['wspeed', 'r2', 't2m']
var_list

In [ ]:
parent_dirs = glob.glob(os.path.join(f'{model}/{p}/', '*'))

In [ ]:
plist = [5,6,7,8]                                                                                                # List of parent directory identifiers
     
for p in plist:                                                                                                  # Loop through each parent directory identifier
    parent_dirs = glob.glob(os.path.join(f'{model}/{p}/', '*'))                                                  # Get all subdirectories in the parent directory
         
    for i in parent_dirs:                                                                                        # Loop through each subdirectory
                 
        dir_maker(os.path.join(f'{output_dir}', f'{i}'))                                                         # Create the corresponding output directory
             
        w_files = glob.glob(os.path.join(f'{output_dir}/{i}/', f'*{var_list[0]}*Abs*'))[0]                       # Find the first file matching the wind speed pattern
        rh_files = glob.glob(os.path.join(f'{output_dir}/{i}/', f'*{var_list[1]}*Abs*'))[0]                      # Find the first file matching the relative humidity pattern
        temp_files = glob.glob(os.path.join(f'{output_dir}/{i}/', f'*{var_list[2]}*Abs*'))[0]                    # Find the first file matching the temperature pattern
         
        ws_file_cc = xr.open_dataset(w_files).chunk(get_chunk(model))                                            # Open and chunk the wind speed file
        rh_file_cc = xr.open_dataset(rh_files).chunk(get_chunk(model))                                           # Open and chunk the relative humidity file
        t2_file_cc = xr.open_dataset(temp_files).chunk(get_chunk(model))                                         # Open and chunk the temperature file
             
        ws_file_cc = ws_file_cc.sortby('valid_time')                                                             # Sort wind speed data by valid time
        rh_file_cc = rh_file_cc.sortby('valid_time')                                                             # Sort relative humidity data by valid time
        t2_file_cc = t2_file_cc.sortby('valid_time')                                                             # Sort temperature data by valid time
        t2_file_cc = (t2_file_cc - 273.15)*(9./5.)+32                                                            # Convert temperature from Kelvin to Fahrenheit
         
        get_m_vec = np.vectorize(get_m)                                                                          # Vectorize the get_m function
        m = xr.apply_ufunc(get_m_vec, rh_file_cc[var_list[1]], t2_file_cc[var_list[2]], dask ='parallelized')    # Broadcast the get_m function to get m from all points on the grid based on their rh and t values
        n = 1-2*(m/30)+1.5*(m/30)**2-0.5*(m/30)**3                                                               # Calculate the n value
        ffwi = (n*((1+ws_file_cc[var_list[0]]**2)**0.5)/0.3002).to_dataset(name = 'ffwi')                        # Calculate the Fire Weather Index (FFWI) and convert to dataset
         
        resampler_UFS('ffwi', ffwi.chunk(get_chunk(model)), output_dir, i, p)                                    # Resample and save the FFWI dataset
     